In [ ]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Tabelas Silver
TB_TRACKS = "spotify_analytics.silver.tb_tracks"
TB_TRACK_ARTISTS = "spotify_analytics.silver.tb_track_artists"

# Dimensões Gold
DM_ARTIST = "spotify_analytics.gold.dm_artist"
DM_TRACK = "spotify_analytics.gold.dm_track"
DM_TEMPO = "spotify_analytics.gold.dm_tempo"

# Fato destino
TABELA_DESTINO = "spotify_analytics.gold.ft_track_popularity"

### Configuração

Define origens (Silver + Dimensões Gold) e destino (Fato Gold).

# Transformação Silver → Gold: ft_track_popularity

## Propósito
Este notebook cria o **fato principal** do Data Warehouse dimensional, consolidando dados de múltiplas fontes em uma **Fact Table otimizada para analytics**.

## O que é uma Fact Table?
Uma tabela fato em um Star Schema contém:
- **Foreign Keys** para dimensões (artist, track, tempo)
- **Métricas** (popularity, duration)
- **Flags de negócio** para filtros rápidos
- **Surrogate Key** (MD5 hash) para garantir unicidade

## Arquitetura Star Schema:

```
dm_artist
     |
     |
dm_track ← ft_track_popularity → dm_tempo
```

## O que este notebook faz:

### 1. Joins com Dimensões
Combina dados Silver com dimensões Gold para obter surrogate keys:
- Silver tb_tracks + tb_track_artists
- JOIN com dm_artist (obter cd_artist)
- JOIN com dm_track (obter cd_track)
- JOIN com dm_tempo (obter cd_tempo)

### 2. Criação de Surrogate Key
- **pk_fato_track**: Hash MD5 de (artist_id + track_id)
- Garante unicidade mesmo com dados distribuídos
- Facilita MERGE (upsert)

### 3. Derivação de Flags de Negócio

**fl_explicit** (0/1):
- Vem diretamente do campo Silver

**fl_alta_popularidade** (0/1):
```sql
fl_alta_popularidade = 1 QUANDO popularity > 70
```
Permite análises como: "Quantos % das tracks são altamente populares?"

**fl_mega_hit** (0/1):
```sql
fl_mega_hit = 1 QUANDO popularity > 90
```
Identifica os maiores sucessos para análise de tendências

**fl_track_curta** (0/1):
```sql
fl_track_curta = 1 QUANDO duration_ms < 180000 (3 minutos)
```
Analisa tendência de encurtamento de músicas ao longo do tempo

## Decisões Técnicas:

### Por que MD5 hash?
- **Distribuição**: Garante distribuição uniforme em Spark
- **Idempotência**: Mesmo input → mesma key
- **Sem Conflitos**: Evita problemas de autoincrement distribuído
- **Performance**: STRING é aceitável para PK em Delta Lake

### Por que INNER JOIN com dimensões?
- **Integridade Referencial**: Garante que toda FK existe na dimensão
- **Dimensões carregadas primeiro**: Processo orquestrado (dim → fato)
- **Sem dados órfãos**: Se artista não está em dm_artist, não deve estar no fato

### Por que flags de negócio?
- **Self-service Analytics**: Analistas podem filtrar sem conhecer regras
- **Performance**: WHERE fl_mega_hit = 1 é mais rápido que WHERE popularity > 90
- **Consistência**: Mesma regra aplicada sempre

## Métricas de Negócio Respondidas:

| Pergunta de Negócio | Como responder |
|---------------------|----------------|
| Qual artista tem mais mega-hits? | GROUP BY cd_artist WHERE fl_mega_hit = 1 |
| Tracks estão ficando mais curtas? | Comparar fl_track_curta por nu_ano (dm_tempo) |
| Qual a popularidade média por década? | JOIN dm_tempo, GROUP BY ds_decada |
| Quantos % das tracks são explícitas? | SUM(fl_explicit) / COUNT(*) * 100 |

## Dependências:
1. Silver carregado (tb_tracks, tb_track_artists)
2. Dimensões Gold carregadas (dm_artist, dm_track, dm_tempo)
3. DDL Gold executado (ddl_ft_track_popularity.ipynb)

## Próximo passo:
Criar views agregadas ou consumir direto no BI.

### 1. Leitura de Tabelas Silver e Dimensões

In [ ]:
# Silver
df_tracks = spark.read.table(TB_TRACKS)
df_track_artists = spark.read.table(TB_TRACK_ARTISTS)

# Dimensões Gold
df_dm_artist = spark.read.table(DM_ARTIST)
df_dm_track = spark.read.table(DM_TRACK)
df_dm_tempo = spark.read.table(DM_TEMPO)

print(f"Tracks: {df_tracks.count()}")
print(f"Track-Artists: {df_track_artists.count()}")
print(f"Dim Artist: {df_dm_artist.count()}")
print(f"Dim Track: {df_dm_track.count()}")
print(f"Dim Tempo: {df_dm_tempo.count()}")

### 2. JOIN Silver: Tracks + Track-Artists

Combina informações de tracks com artistas.

In [ ]:
df_track_full = (
    df_tracks.alias("t")
    .join(df_track_artists.alias("ta"), on="track_id", how="inner")
    .select(
        "t.track_id",
        "ta.artist_id",
        "t.popularity",
        "t.duration_ms",
        "t.explicit",
        "t.release_date"
    )
)

print(f"Registros track-artist combinados: {df_track_full.count()}")

### 3. JOIN com Dimensões para obter Surrogate Keys

**Importante**: Usamos INNER JOIN para garantir integridade referencial.
Se um artista/track/data não existe na dimensão, não deve estar no fato.

In [ ]:
# JOIN com dm_artist
df_with_artist = (
    df_track_full.alias("f")
    .join(
        df_dm_artist.alias("da"),
        F.col("f.artist_id") == F.col("da.id_artist_spotify"),
        how="inner"
    )
    .select("f.*", F.col("da.cd_artist"))
)

# JOIN com dm_track
df_with_track = (
    df_with_artist.alias("f")
    .join(
        df_dm_track.alias("dt"),
        F.col("f.track_id") == F.col("dt.id_track_spotify"),
        how="inner"
    )
    .select("f.*", F.col("dt.cd_track"))
)

# JOIN com dm_tempo (usando release_date)
df_with_tempo = (
    df_with_track.alias("f")
    .join(
        df_dm_tempo.alias("dtemp"),
        F.col("f.release_date") == F.col("dtemp.dt_completa"),
        how="inner"
    )
    .select("f.*", F.col("dtemp.cd_tempo"))
)

print(f"Registros após JOINs com dimensões: {df_with_tempo.count()}")

### 4. Criação do Fato com Surrogate Key e Flags de Negócio

In [ ]:
df_fato = df_with_tempo.select(
    # Surrogate key: MD5 hash de artist_id + track_id
    F.md5(F.concat(
        F.col("artist_id"),
        F.lit("|"),
        F.col("track_id")
    )).alias("pk_fato_track"),
    
    # Foreign keys (surrogate)
    F.col("cd_artist"),
    F.col("cd_track"),
    F.col("cd_tempo"),
    
    # Métricas
    F.col("popularity").alias("val_popularity"),
    F.col("duration_ms").alias("val_duration_ms"),
    
    # Flags de negócio
    F.when(F.col("explicit") == True, 1).otherwise(0).cast("smallint").alias("fl_explicit"),
    
    F.when(F.col("popularity") > 70, 1).otherwise(0).cast("smallint").alias("fl_alta_popularidade"),
    
    F.when(F.col("popularity") > 90, 1).otherwise(0).cast("smallint").alias("fl_mega_hit"),
    
    F.when(F.col("duration_ms") < 180000, 1).otherwise(0).cast("smallint").alias("fl_track_curta")
)

print(f"Total de registros no fato: {df_fato.count()}")
df_fato.show(10, truncate=False)

### 5. MERGE no Fato

**Decisão Arquitetural**: Usar MERGE ao invés de OVERWRITE:
- **Vantagem**: Suporta cargas incrementais futuras
- **Vantagem**: Preserva histórico se implementarmos versionamento
- **Trade-off**: Mais lento que OVERWRITE, mas mais robusto

In [ ]:
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_fato.alias("s"),
    "t.pk_fato_track = s.pk_fato_track"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Fato Track Popularity carregado!")

### 6. Verificações de Qualidade

In [ ]:
# Total de registros
total = spark.table(TABELA_DESTINO).count()
print(f"\nTotal de registros no fato: {total}")

# Distribuição de flags
print("\n=== Distribuição de Flags de Negócio ===")
spark.sql(f"""
    SELECT 
        SUM(fl_explicit) as qtd_explicit,
        SUM(fl_alta_popularidade) as qtd_alta_popularidade,
        SUM(fl_mega_hit) as qtd_mega_hits,
        SUM(fl_track_curta) as qtd_tracks_curtas,
        COUNT(*) as total
    FROM {TABELA_DESTINO}
""").show()

# Percentuais
print("\n=== Percentuais ===")
spark.sql(f"""
    SELECT 
        ROUND(SUM(fl_explicit) / COUNT(*) * 100, 2) as pct_explicit,
        ROUND(SUM(fl_alta_popularidade) / COUNT(*) * 100, 2) as pct_alta_pop,
        ROUND(SUM(fl_mega_hit) / COUNT(*) * 100, 2) as pct_mega_hits,
        ROUND(SUM(fl_track_curta) / COUNT(*) * 100, 2) as pct_curtas
    FROM {TABELA_DESTINO}
""").show()

# Top artistas com mais mega-hits (JOIN com dimensão)
print("\n=== Top 5 Artistas com Mais Mega-Hits ===")
spark.sql(f"""
    SELECT 
        a.nm_artist,
        SUM(f.fl_mega_hit) as qtd_mega_hits,
        AVG(f.val_popularity) as avg_popularity
    FROM {TABELA_DESTINO} f
    INNER JOIN {DM_ARTIST} a ON f.cd_artist = a.cd_artist
    WHERE f.fl_mega_hit = 1
    GROUP BY a.nm_artist
    ORDER BY qtd_mega_hits DESC
    LIMIT 5
""").show(truncate=False)